# Import Required Libraries

In [22]:
import bs4,lxml,requests
import re
# pip install google
from googlesearch import search 
from xlwt import Workbook 
punctuations = '!"#$%&\'()*,/:;<=>?[\\]^_`{|}~'

# Get Website URL and Number of pages to Scrape

In [6]:
url = input("Enter URL : ").strip()

if url == '':
    print("Please Enter Valide URL")
    
numberOfWebpageToScrape = input("Number Of Webpage To Scrape : ")
if numberOfWebpageToScrape.isdigit():
    numberOfWebpageToScrape = int(numberOfWebpageToScrape)
else:
    numberOfWebpageToScrape = 1
print(numberOfWebpageToScrape)

Enter URL : https://argusoft.com/
Number Of Webpage To Scrape : 1
1


# Invalide Resources Extensions

In [7]:
invalidResourceExtensions = [".WEBM",".JPEG",".JPG", ".PNG", ".GIF", ".TIFF", ".PSD",".PDF",".EPS", 
    ".AI",".INDD",".RAW", ".pdf",".epub",".7z",".rar",".zip",".tar",".OGG",".MP4",".M4P", ".M4V",
    ".AVI", ".WMV", ".MOV", ".QT", ".FLV", ".SWF", ".json", ".svg", ".css", ".js"
]

for ire in range(len(invalidResourceExtensions)):
    invalidResourceExtensions[ire] = invalidResourceExtensions[ire].lower()
print("invalidResourceExtensions :")
print(invalidResourceExtensions)

invalidResourceExtensions :
['.webm', '.jpeg', '.jpg', '.png', '.gif', '.tiff', '.psd', '.pdf', '.eps', '.ai', '.indd', '.raw', '.pdf', '.epub', '.7z', '.rar', '.zip', '.tar', '.ogg', '.mp4', '.m4p', '.m4v', '.avi', '.wmv', '.mov', '.qt', '.flv', '.swf', '.json', '.svg', '.css', '.js']


# Punctuation remover

In [8]:
# punctuations are removed for better searching
def removePunctuation(wordsList): 
    for word in range(len(wordsList)):
        for char in wordsList[word]:
            if char in punctuations:
                 wordsList[word] = wordsList[word].replace(char, '')
    return wordsList 

# Find Emails from webpages

In [9]:
emailRegexPattern = "^([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})$"
emailRegex = re.compile(emailRegexPattern)

# find Email
def findEmails(wordsList):
    # Valid email list
    emails = list()
    
    for word in wordsList:
        # checking for string and handles error created by other types
        if type(word) is str and '@' in word:
            if emailRegex.match(word):
                emails.append(word)
    return emails

# Find Mobiles Number from webpages

In [10]:
# mobile Regex configurations
mobileRegexPatter = """^([+]?[0-9]+)?( ?[0-9]+)?(-?[0-9]+)*$"""
mobileRegex = re.compile(mobileRegexPatter)

# Find Mobile Number
def findMobileNumber(words):
    mobileNumber = list()
    
    for word in words:
        # We already sorted the list 
        # so first encounter of alpha will terminate the loop
        if word[0].isalpha():
            return mobileNumber
        else:
            if mobileRegex.match(word) and len(word) > 5:
                mobileNumber.append(word)
    return mobileNumber

# Clean Data

In [11]:
def cleanData(beautifiedData):
    # get all text from response
    # remove "\n" and split using space
    words = beautifiedData.text.replace('\n', ' ').split(' ')
    print("Text Length Before removing redundant words : ", len(words))

    # clean text by
    # removing redundant words
    cleanedWords = list(set(words))
    print("Text Length After removing redundant words : ", len(cleanedWords))

    # removing punctuation and redundant words
    cleanedWords = list(set(removePunctuation(cleanedWords)))
    print("Text Length After removing punctuations from words : ", len(cleanedWords))

    # sort cleanedWords
    cleanedWords.sort()

    # removing empty string
    if cleanedWords[0] == '':
        cleanedWords.remove('')
    return cleanedWords

# Find All URLs from webpage

In [18]:
def getUrlsFromWebPage(beautifiedData):
    hrefTags = beautifiedData.select('[href]')
    validURLs = list()

    for hrefTag in hrefTags:
        if hrefTag['href'].startswith(url):
            for extensions in invalidResourceExtensions:
                if hrefTag['href'].endswith(extensions):
                    break
            else:
                validURLs.append(hrefTag['href'])
    
    print("Total ValidURLs : ", len(validURLs)) 
    return validURLs

# Scrapping Webpage

In [13]:
def scrapeWebpage(websiteURL):
    response = requests.get(url)
    beautifiedData = bs4.BeautifulSoup(response.text, "lxml")
    return beautifiedData

In [15]:
def scrapeWebSite(websiteURL, numberOfWebpageToScrape):   
    beautifiedData = scrapeWebpage(websiteURL)
    data = dict()
    validURLs = getUrlsFromWebPage(beautifiedData)[0:numberOfWebpageToScrape]
    
    # add websiteURL's data
    data[websiteURL] = {
                "email" : findEmails(cleanData(beautifiedData)),
                "mobile" : findMobileNumber(cleanData(beautifiedData))
        }
    
    for url in validURLs:
        print("\n\nStarted Scrapping : ", url, "\n")
        tempScrappedData = scrapeWebpage(url)
        cleanedData = cleanData(tempScrappedData)
        data[url] = {
                "email" : findEmails(cleanedData),
                "mobile" : findMobileNumber(cleanedData)
        }
    
    return data

# scrapeWebSite(url, numberOfWebpageToScrape)

Total ValidURLs :  0
Text Length Before removing redundant words :  10659
Text Length After removing redundant words :  1304
Text Length After removing punctuations from words :  1240
Text Length Before removing redundant words :  10659
Text Length After removing redundant words :  1304
Text Length After removing punctuations from words :  1240


{'https://argusoft.com/': {'email': ['info@argusoft.com', 'jobs@argusoft.com'],
  'mobile': ['+1-510-435-0567', '9328573110']}}

# Create Excel File

In [40]:
def createExcelFile(data):
    trackURLrow = 1
    # Workbook is created 
    wb = Workbook() 
    tempSheet = wb.add_sheet('data') 
    tempSheet.write(0,0,"URL")
    tempSheet.write(0,1,"E-mail")
    tempSheet.write(0,2,"Mobile Number")
    
    for url in data:
        tempSheet.write(trackURLrow, 0, url) 
        
        for mail in range(len(data[url]['email'])):
            print(trackURLrow + mail, 1, data[url]['email'][mail])
            tempSheet.write(trackURLrow + mail, 1, data[url]['email'][mail])
        
        for mobile in range(len(data[url]['mobile'])):
            print(trackURLrow + mobile, 1, data[url]['email'][mail])
            tempSheet.write(trackURLrow + mobile, 2, data[url]["mobile"][mobile])
        
        trackURLrow += max(len(data[url]['email']), len(data[url]['mobile']), 1) + 1 
            

    wb.save('xlwt example.xls') 
createExcelFile(data)

1 1 info@argusoft.com
2 1 jobs@argusoft.com
1 1 jobs@argusoft.com
2 1 jobs@argusoft.com
6 1 rgopalan@argusoft.com
6 1 rgopalan@argusoft.com
7 1 rgopalan@argusoft.com
8 1 rgopalan@argusoft.com
9 1 rgopalan@argusoft.com
10 1 rgopalan@argusoft.com
11 1 rgopalan@argusoft.com
12 1 rgopalan@argusoft.com
13 1 rgopalan@argusoft.com
14 1 rgopalan@argusoft.com
15 1 rgopalan@argusoft.com


In [19]:
query = input("Enter search text : ")
data = {}

for url in search(query, tld="co.in", num=5, stop=5, pause=2): 
    print(url)
    data.update(scrapeWebSite(url,numberOfWebpageToScrape))



Enter search text : argusoft.com
https://argusoft.com/
Total ValidURLs :  0
Text Length Before removing redundant words :  10659
Text Length After removing redundant words :  1304
Text Length After removing punctuations from words :  1240
Text Length Before removing redundant words :  10659
Text Length After removing redundant words :  1304
Text Length After removing punctuations from words :  1240
https://www.linkedin.com/company/argusoft
Total ValidURLs :  0
Text Length Before removing redundant words :  3
Text Length After removing redundant words :  1
Text Length After removing punctuations from words :  1
Text Length Before removing redundant words :  3
Text Length After removing redundant words :  1
Text Length After removing punctuations from words :  1
https://www.zaubacorp.com/company/ARGU-SOFT-INDIA-LIMITED/U74899GJ2000PLC064126
Total ValidURLs :  3
Text Length Before removing redundant words :  2576
Text Length After removing redundant words :  690
Text Length After removing

In [20]:
print(data)

{'https://argusoft.com/': {'email': ['info@argusoft.com', 'jobs@argusoft.com'],
  'mobile': ['+1-510-435-0567', '9328573110']},
 'https://www.linkedin.com/company/argusoft': {'email': [], 'mobile': []},
 'https://www.zaubacorp.com/company/ARGU-SOFT-INDIA-LIMITED/U74899GJ2000PLC064126': {'email': ['rgopalan@argusoft.com'],
  'mobile': ['00301915',
   '00948595',
   '01029288',
   '1000000',
   '17948000',
   '2005-02-08',
   '2016-10-19',
   '382010',
   '382016',
   '90039792']},
 'https://www.glassdoor.co.in/Reviews/Argusoft-Reviews-E754272.htm': {'email': [],
  'mobile': []},
 'https://www.glassdoor.co.in/Overview/Working-at-Argusoft-EI_IE754272.11,19.htm': {'email': [],
  'mobile': []}}